In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import pandas as pd
import os

In [3]:
# variables to be modified by the user
    # input paths
VIBRANT_output_path = '/home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/'
project_root = '/home/abelardoacm/Storage/Abelardo/projects/mini-devel/'
JGI_global_metadata_csv_path = project_root + 'data/1_environmental_seqs_from_JGI_IMGvir/DTRs_20kb.csv'
    # parameters
min_quality_by_usr = "high" # 'low', 'medium', 'high', 'complete'
min_contig_length_by_usr = 0 # in bp
max_contig_length_by_usr = 5000 # in bp
mcp_terms_by_usr = None #(list, optional): Terms related to MCP. Defaults to a predefined list if None.
false_terms_by_usr = None #(list, optional): Terms to identify and exclude MPC false positives. Defaults to a predefined list if None.
    # output paths
        # NOTE: the output paths are created automatically within VIBRAnt_output_path
        #       and are named as follows:
mcps_fasta_suffix = '_MCPs.faa'


In [4]:
# create the output directory if it does not exist
output_directory = VIBRANT_output_path + '2_filtered_VIBRANT_output/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
os.chdir(project_root)
VIBRANT_output_path_elements = VIBRANT_output_path.split('/')
VIBRANT_folder_name = next((element for element in reversed(VIBRANT_output_path_elements) if element), None)
VIBRANT_input_name = '_'.join(VIBRANT_folder_name.split('_')[1:])

In [5]:
# load the JGI global metadata
JGI_global_metadata = pd.read_csv(JGI_global_metadata_csv_path)

# See VIBRANT Output explanations for more information on the output files and numbers
# output 38 is the VIBRANT annotation file
output38_file_path =  VIBRANT_output_path + 'VIBRANT_results_' + VIBRANT_input_name +'/VIBRANT_annotations_' + VIBRANT_input_name + '.tsv'
VIBRANT_annotations = pd.read_csv(output38_file_path, sep='\t')

# output 42 is the list of predicted genome quality and type
output42_file_path =  VIBRANT_output_path + 'VIBRANT_results_' + VIBRANT_input_name +'/VIBRANT_genome_quality_' + VIBRANT_input_name + '.tsv'
VIBRANT_genome_quality = pd.read_csv(output42_file_path, sep='\t')

# output 23 corresponds to all encoded proteins among identified phages
output23_file_path =  VIBRANT_output_path + 'VIBRANT_phages_' + VIBRANT_input_name + '/' + VIBRANT_input_name + '.phages_combined.faa'
from src.auxiliary_functions.VIBRANT_filtering_functions import read_VIBRANT_custom_fasta_23
VIBRANT_phages_proteins = read_VIBRANT_custom_fasta_23(output23_file_path)
count_VIBRANT_phages_proteins = len(VIBRANT_phages_proteins)

# output 25 corresponds to all phages genomes
output23_file_path =  VIBRANT_output_path + 'VIBRANT_phages_' + VIBRANT_input_name + '/' + VIBRANT_input_name + '.phages_combined.fna'
VIBRANT_phages_genomes = SeqIO.parse(output23_file_path, "fasta")
VIBRANT_phages_genomes_as_list = list(SeqIO.parse(output23_file_path, "fasta"))
count_VIBRANT_phages_genomes = sum(1 for _ in VIBRANT_phages_genomes)

print('count_VIBRANT_phages_genomes:', count_VIBRANT_phages_genomes)
print('count_VIBRANT_phages_proteins:', count_VIBRANT_phages_proteins)

count_VIBRANT_phages_genomes: 5711
count_VIBRANT_phages_proteins: 94143


In [6]:
from src.auxiliary_functions.VIBRANT_filtering_functions import filter_VIBRANT_annotations_by_mcp
# Filter the VIBRANT annotations DataFrame based on MCP-related terms
filtered_VIBRANT_contigs_w_mcp = filter_VIBRANT_annotations_by_mcp(VIBRANT_annotations,mcp_terms=mcp_terms_by_usr, false_terms=false_terms_by_usr)
filtered_VIBRANT_contigs_w_mcp_path = output_directory + VIBRANT_input_name + '_contigs_w_mcp.tsv'
filtered_VIBRANT_contigs_w_mcp.to_csv(filtered_VIBRANT_contigs_w_mcp_path, sep='\t', index=False)
print('Contigs with MCP DataFrame written to:', filtered_VIBRANT_contigs_w_mcp_path)

from src.auxiliary_functions.VIBRANT_filtering_functions import write_VIBRANT_contigs_w_mcp_fasta
write_VIBRANT_contigs_w_mcp_fasta(filtered_VIBRANT_contigs_w_mcp, output23_file_path, output_directory, VIBRANT_input_name,VIBRANT_phages_proteins)


from src.auxiliary_functions.VIBRANT_filtering_functions import filter_and_write_MCP_sequences
# Filter the VIBRANT proteins and write the MCP sequences to a fasta file
filtered_MCP_sequences = filter_and_write_MCP_sequences(filtered_VIBRANT_contigs_w_mcp, VIBRANT_phages_proteins, output_directory)


Contigs with MCP DataFrame written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_contigs_w_mcp.tsv

From the file: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_contigs_with_mcp.fna
	Number of genomes: 834 Number of matching proteins: 14671
	Matching proteins written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_contigs_with_mcp.faa
834 contigs with MCP fasta (.fna) written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_contigs_with_mcp.fna

From the file: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DT

In [7]:
from src.auxiliary_functions.VIBRANT_filtering_functions import filter_VIBRANT_genome_quality
# Filter the VIBRANT genome quality DataFrame based on the completeness of the genomes
filtered_VIBRANT_genome_quality = filter_VIBRANT_genome_quality(VIBRANT_genome_quality, min_quality_by_usr)
# Write the filtered VIBRANT genome quality DataFrame to a csv file
filtered_VIBRANT_genome_quality_path = output_directory + VIBRANT_input_name + '_minimum_' + min_quality_by_usr + '_quality_genomes_' + '.csv'
filtered_VIBRANT_genome_quality.to_csv(filtered_VIBRANT_genome_quality_path, index=False)
print('Filtered VIBRANT genome quality DataFrame written to:', filtered_VIBRANT_genome_quality_path)

# Write the phage genomes filtered by genome quality to a fasta file
from src.auxiliary_functions.VIBRANT_filtering_functions import filter_and_write_genomes_by_quality
filter_and_write_genomes_by_quality(VIBRANT_phages_genomes, filtered_VIBRANT_genome_quality, min_quality_by_usr, output_directory, VIBRANT_input_name,output23_file_path,VIBRANT_phages_proteins)



Filtered VIBRANT genome quality DataFrame written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_minimum_high_quality_genomes_.csv
5694(out of 5711) contigs of at least high quality (.fna) written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_above_high_quality_genomes_.fna
17(out of 5711) contigs below high quality (.fna) written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_bellow_high_quality_genomes_.fna

From the file: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_above_high_quality_genomes_.fna
	Number of genomes: 5694 Number of matching proteins: 93878
	Matching proteins written to: 

In [8]:
from src.auxiliary_functions.VIBRANT_filtering_functions import filter_scaffolds_by_contig_length
# Filter the JGI global metadata DataFrame based on the length of the contigs
filtered_JGI_global_metadata = filter_scaffolds_by_contig_length(JGI_global_metadata, min_contig_length=min_contig_length_by_usr, max_contig_length=max_contig_length_by_usr)
filtered_JGI_global_metadata_path = output_directory + 'JGI_global_metadata_filtered_from_'+ str(min_contig_length_by_usr) + 'kb_to_' + str(max_contig_length_by_usr) + 'kb_' + VIBRANT_input_name + '.csv'
filtered_JGI_global_metadata.to_csv(filtered_JGI_global_metadata_path, index=False)
print('Filtered JGI global metadata DataFrame written to:', filtered_JGI_global_metadata_path)

from src.auxiliary_functions.VIBRANT_filtering_functions import write_contigs_by_contig_length

write_contigs_by_contig_length(VIBRANT_phages_genomes, JGI_global_metadata, filtered_JGI_global_metadata, output_directory, VIBRANT_input_name,output23_file_path,VIBRANT_phages_proteins)


Filtered JGI global metadata DataFrame written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/JGI_global_metadata_filtered_from_0kb_to_5000kb_DTRs_20kb.csv
755(out of initial 7650) contigs within 2001kb to 5000kb written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_contigs_within_2001kb_to_5000kb_.fna
4956(out of initial 7650) contigs NOT within 2001kb to 5000kb written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_contigs_within_5002kb_to_19990kb_.fna

From the file: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/DTRs_20kb_contigs_within_2001kb_to_5000kb_.fna
	Number of genomes: 755 Number of matching protein

In [9]:
#Find phages meeting all the criteria

# Aquire contigs ids with MCPs
contigs_w_mcp_ids = filtered_VIBRANT_contigs_w_mcp['scaffold'].tolist()
print('Number of contigs with MCPs:', len(contigs_w_mcp_ids))
# Aquire contigs ids with sufficient quality
contigs_w_sufficient_quality_ids = filtered_VIBRANT_genome_quality['scaffold'].tolist()
print('Number of contigs with sufficient quality:', len(contigs_w_sufficient_quality_ids))
# Aquire contigs ids within length range
print('Number of contigs within length range:', len(filtered_JGI_global_metadata))
contigs_w_length = filtered_JGI_global_metadata['genome_id'].tolist()

# Find the intersection of pairs of lists
phages_meeting_mcp_and_quality_criteria = list(set(contigs_w_mcp_ids) & set(contigs_w_sufficient_quality_ids))
print('Number of phages meeting MCP and quality criteria:', len(phages_meeting_mcp_and_quality_criteria))
phages_meeting_mcp_and_length_criteria = list(set(contigs_w_mcp_ids) & set(contigs_w_length))
print('Number of phages meeting MCP and length criteria:', len(phages_meeting_mcp_and_length_criteria))
phages_meeting_quality_and_length_criteria = list(set(contigs_w_sufficient_quality_ids) & set(contigs_w_length))
print('Number of phages meeting quality and length criteria:', len(phages_meeting_quality_and_length_criteria))

# Create paths for the pairs of lists
phages_meeting_mcp_and_quality_criteria_path = output_directory + 'phages_meeting_mcp_and_quality_criteria_' + VIBRANT_input_name + '.fna'
phages_meeting_mcp_and_length_criteria_path = output_directory + 'phages_meeting_mcp_and_length_criteria_' + VIBRANT_input_name + '.fna'
phages_meeting_quality_and_length_criteria_path = output_directory + 'phages_meeting_quality_and_length_criteria_' + VIBRANT_input_name + '.fna'

# Save the genomes phages meeting the pairs of criteria to .fna files
with open(phages_meeting_mcp_and_quality_criteria_path, 'w') as f:
    for genome in VIBRANT_phages_genomes_as_list:
        if genome.id in phages_meeting_mcp_and_quality_criteria:
            SeqIO.write(genome, f, "fasta")
print('Phages meeting MCP and quality criteria written to:', phages_meeting_mcp_and_quality_criteria_path)

with open(phages_meeting_mcp_and_length_criteria_path, 'w') as f:
    for genome in VIBRANT_phages_genomes_as_list:
        if genome.id in phages_meeting_mcp_and_length_criteria:
            SeqIO.write(genome, f, "fasta")
print('Phages meeting MCP and length criteria written to:', phages_meeting_mcp_and_length_criteria_path)

with open(phages_meeting_quality_and_length_criteria_path, 'w') as f:
    for genome in VIBRANT_phages_genomes_as_list:
        if genome.id in phages_meeting_quality_and_length_criteria:
            SeqIO.write(genome, f, "fasta")
print('Phages meeting quality and length criteria written to:', phages_meeting_quality_and_length_criteria_path)


# Find the intersection of the three lists
phages_meeting_all_criteria = list(set(contigs_w_mcp_ids) & set(contigs_w_sufficient_quality_ids) & set(contigs_w_length))
print('Number of phages meeting all the criteria:', len(phages_meeting_all_criteria))

# Save the genomes phages meeting all the criteria to a .fna file
phages_meeting_all_criteria_path = output_directory + 'phages_meeting_all_criteria_' + VIBRANT_input_name + '.fna'

with open(phages_meeting_all_criteria_path, 'w') as f:
    for genome in VIBRANT_phages_genomes_as_list:
        if genome.id in phages_meeting_all_criteria:
            SeqIO.write(genome, f, "fasta")
print('Phages meeting all the criteria written to:', phages_meeting_all_criteria_path)

from src.auxiliary_functions.VIBRANT_filtering_functions import search_identifiers_in_VIBRANT_phages_proteins
search_identifiers_in_VIBRANT_phages_proteins(phages_meeting_all_criteria_path, VIBRANT_phages_proteins)
search_identifiers_in_VIBRANT_phages_proteins(phages_meeting_mcp_and_quality_criteria_path, VIBRANT_phages_proteins)
search_identifiers_in_VIBRANT_phages_proteins(phages_meeting_mcp_and_length_criteria_path, VIBRANT_phages_proteins)
search_identifiers_in_VIBRANT_phages_proteins(phages_meeting_quality_and_length_criteria_path, VIBRANT_phages_proteins)


Number of contigs with MCPs: 864
Number of contigs with sufficient quality: 5701
Number of contigs within length range: 1495
Number of phages meeting MCP and quality criteria: 831
Number of phages meeting MCP and length criteria: 11
Number of phages meeting quality and length criteria: 755
Phages meeting MCP and quality criteria written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/phages_meeting_mcp_and_quality_criteria_DTRs_20kb.fna
Phages meeting MCP and length criteria written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/phages_meeting_mcp_and_length_criteria_DTRs_20kb.fna
Phages meeting quality and length criteria written to: /home/abelardoacm/Storage/Abelardo/projects/mini-devel/results/1_biome_fasta_files/biome0/VIBRANT_DTRs_20kb/2_filtered_VIBRANT_output/phages_meeting_quality_and_length_criteria

[SeqRecord(seq=Seq('MAVSKRARARFKIMSASEKAAVRKAVKLLYDTELVGIKRMREITRLAEKR*'), id='DTR_893050_1', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MPVPNYNSWIQSQGEITAGSDALVVFEIFSAAEKPVRVEQMHYFGGDAGEFMQL...DA*'), id='DTR_893050_2', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MPKSAPTQVIVHRIEFQESEREILKDVAMTWQATRIAQPIVALINDNTTLLLIL...EP*'), id='DTR_893050_3', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MKNWVIKSISIEHSDAEIIRMQGQGFNLSRFVRVCLRRHALHQEQLSMIHRQPG...DP*'), id='DTR_893050_4', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MKCVELWSGDSTGLEAARRRGHTVLTVDNDPKFNADICADIATVTADQIRVALN...WL*'), id='DTR_893050_5', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MAVTQAPLEIKYLRDAWMGKRDTHETRYFAYIWKEITAMDFWVDNAVIVDPFAR...GA*'), id='DTR_893050_6', name='<unknown name>', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('MEWVLVCVVAVIALQLATIKILFEMVQRISISIDVVGHSLKKIDSMIEMTGIAG...KE